In [14]:
import os, sys
import gc
import time

import torch
gc.collect()
# os.environ['CUDA_LAUNCH_BLOCKING'] = "4"
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"  
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
torch.cuda.set_device(device)
# print(device)

from sklearn.metrics import accuracy_score
import swifter

from tqdm.notebook import tqdm
tqdm.pandas()
from nltk.tokenize import word_tokenize


from utils.utils_init_dataset import set_seed, load_dataset_loader
from utils.utils_semantic_use import USE
from utils.utils_data_utils import DocumentSentimentDataset, DocumentSentimentDataLoader, EmotionDetectionDataset, EmotionDetectionDataLoader
from utils.utils_metrics import document_sentiment_metrics_fn
from utils.utils_init_model import text_logit, fine_tuning_model, eval_model, init_model, logit_prob, load_word_index
from utils.get_args import get_args

# from attack.adv_attack import attack
import os, sys

from utils.utils_init_model import text_logit, fine_tuning_model, eval_model, init_model, logit_prob, load_word_index
import time
from nltk.tokenize import word_tokenize
import torch
import torch.nn.functional as F
import numpy as np
import math
from attack.attack_helper import get_synonyms, codemix_perturbation, synonym_replacement, swap_minimum_importance_words

from attack.adv_attack import attack
import main

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words_set = []
for w in stopwords.words('indonesian'):
    stop_words_set.append(w)

from icecream import ic
import pandas as pd
pd.set_option('display.max_colwidth', None)
from itertools import chain
from deep_translator import GoogleTranslator


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/m13518040/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# def trans_dict(wordlist, perturbation_technique, lang=None):
#     translated_wordlist = {}
#     # ic(wordlist)
#     if perturbation_technique == "codemixing":
#         translator = GoogleTranslator(source="id", target=lang)
        
#         for word in wordlist:
#             if word.isalpha():
#                 translated_wordlist[word] = translator.translate(word)
#             else:
#                 translated_wordlist[word] = word
#     else:
#         for word in wordlist:
#             translated_wordlist[word] = get_synonyms(word)[0]
            
#     return translated_wordlist


# def attack(text_ls,
#            true_label,
#            predictor,
#            tokenizer,
#            att_ratio,
#            perturbation_technique,
#            lang_codemix=None,
#            sim_predictor=None,
#            sim_score_threshold=0.5,
#            sim_score_window=15,
#            batch_size=32, 
#            import_score_threshold=-1.):
    
#     start_time = time.time()
    
#     label_dict = {
#         'positive': 0, 
#         'neutral': 1, 
#         'negative': 2}
    
#     original_text = text_ls
#     orig_label, orig_probs, orig_prob = logit_prob(text_ls, predictor, tokenizer)
        
# #     SEK SALAAHHHHHH
#     if true_label != orig_label:
#         running_time = round(time.time() - start_time, 2)
#         # perturbed_text, perturbed_semantic_sim, orig_label, orig_probs, perturbed_label, perturbed_probs, translated_words, running_time
#         return original_text, 1.000, orig_label, orig_probs, orig_label, orig_probs,'', running_time
#     else:
#         text_ls = word_tokenize(text_ls)
#         text_ls = [word for word in text_ls if word.isalnum()]
#         len_text = len(text_ls)
#         half_sim_score_window = (sim_score_window - 1) // 2
        
#         leave_1_texts = [' '.join(text_ls[:ii] + [tokenizer.mask_token] + text_ls[min(ii + 1, len_text):]) for ii in range(len_text)]
                
#         leave_1_probs = []
#         leave_1_probs_argmax = []
#         for text_leave_1 in leave_1_texts:
#             subwords_leave_1 = tokenizer.encode(text_leave_1)
#             subwords_leave_1 = torch.LongTensor(subwords_leave_1).view(1, -1).to(predictor.device)
#             logits_leave_1 = predictor(subwords_leave_1)[0]
#             orig_label_leave_1 = torch.topk(logits_leave_1, k=1, dim=-1)[1].squeeze().item()
            
#             leave_1_probs_argmax.append(orig_label_leave_1)
#             leave_1_probs.append(F.softmax(logits_leave_1, dim=-1).squeeze().detach().cpu().numpy())
            
#         leave_1_probs = torch.tensor(leave_1_probs).to(device)
        
#         orig_prob_extended=np.empty(len_text)
#         orig_prob_extended.fill(orig_prob)
#         orig_prob_extended = torch.tensor(orig_prob_extended).to(device)
        
#         arr1 = orig_prob_extended - leave_1_probs[:,orig_label] + float(leave_1_probs_argmax != orig_label)
#         arr2 = (leave_1_probs.max(dim=-1)[0].to(device) - orig_probs[leave_1_probs_argmax].to(device))
        
#         import_scores = arr1*arr2
#         import_scores = [im * -1 for im in import_scores]
        
#         words_perturb = []
#         for idx, score in sorted(enumerate(import_scores), key=lambda x: x[1], reverse=True):
#             try:
#                 if score > import_score_threshold and text_ls[idx] not in stop_words_set:
#                     words_perturb.append((idx, text_ls[idx], score.item()))
#             except Exception as e:
#                 print(e)
#                 print(idx, len(text_ls), import_scores.shape, text_ls, len(leave_1_texts))
        
#         num_perturbation = math.floor(len(words_perturb)*att_ratio)
#         if num_perturbation < 1:
#             num_perturbation = 1
        
# #       top words perturb berisi list kata terpenting yang tidak akan diswitch ketika first_codemix_sim_score < sim_score_threshold
#         top_words_perturb = words_perturb[:num_perturbation]
#         trans_word = [twp[1] for twp in top_words_perturb]

#         if perturbation_technique == "codemixing":
#             perturbed_text = codemix_perturbation(text_ls, lang_codemix, top_words_perturb)
#         elif perturbation_technique == "synonym_replacement":
#             perturbed_text = synonym_replacement(text_ls, top_words_perturb)
        
#         first_perturbation_sim_score = sim_predictor.semantic_sim(original_text, perturbed_text)

#         words_perturb_candidates = []
#         if first_perturbation_sim_score < sim_score_threshold:
#             words_perturb_candidates.append(top_words_perturb)
#             swapped = swap_minimum_importance_words(words_perturb, top_words_perturb)
#             for s in swapped:
#                 words_perturb_candidates.append(s)

#             words_perturb_candidates = [[tuple(w) for w in wpc] for wpc in words_perturb_candidates]

#             candidate_comparison = {}
#             for wpc in words_perturb_candidates:
#                 # ic(wpc)
#                 if perturbation_technique == "codemixing":
#                     perturbed_candidate = codemix_perturbation(text_ls, lang_codemix, wpc)
#                 elif perturbation_technique == "synonym_replacement":
#                     perturbed_candidate = synonym_replacement(text_ls, wpc)
                    
#                 perturbed_candidate_sim_score = sim_predictor.semantic_sim(original_text, perturbed_candidate)
#                 # ic(wpc[-1][-1])
#                 candidate_comparison[perturbed_candidate] = (perturbed_candidate_sim_score, wpc[-1][-1], wpc)

#             ic(candidate_comparison)
#             sorted_candidate_comparison = sorted(candidate_comparison.keys(), key=lambda x: (candidate_comparison[x][0], candidate_comparison[x][1]), reverse=True)

#             perturbed_text = sorted_candidate_comparison[0]
            
#             top_words_perturb = candidate_comparison[sorted_candidate_comparison[0]][-1]
#             trans_word = [twp[1] for twp in top_words_perturb]
            
#         perturbed_semantic_sim = sim_predictor.semantic_sim(original_text, perturbed_text)
#         if perturbed_semantic_sim < sim_score_threshold:
#             perturbed_text = original_text
#             perturbed_semantic_sim = 1.000
        
#         perturbed_label, perturbed_probs, perturbed_prob = logit_prob(perturbed_text, predictor, tokenizer)
        
        
#         translated_words = trans_dict(trans_word, perturbation_technique, lang_codemix)
        
#         orig_probs = np.round(orig_probs.detach().cpu().numpy().tolist(),4)
#         perturbed_probs = np.round(perturbed_probs.detach().cpu().numpy().tolist(),4)
        
#         running_time = round(time.time() - start_time, 2)
        
#         ic(perturbed_text)
#         ic(perturbed_semantic_sim)
#         ic(orig_label)
#         ic(orig_probs)
#         ic(perturbed_label)
#         ic(perturbed_probs)
#         ic(translated_words)
#         ic(running_time)
        
        
#         return perturbed_text, perturbed_semantic_sim, orig_label, orig_probs, perturbed_label, perturbed_probs, translated_words, running_time

In [7]:
model_target="IndoBERT"
downstream_task="sentiment"
attack_strategy="adversarial"
finetune_epoch=1
num_sample=2
# exp_name=
perturbation_technique="codemixing"
perturb_ratio=0.4
perturb_lang="en"
seed=26092020

In [3]:
set_seed(seed)
use = USE()

tokenizer, config, model = init_model(model_target, downstream_task)
w2i, i2w = load_word_index(downstream_task)

train_dataset, train_loader, train_path = load_dataset_loader(downstream_task, 'train', tokenizer)
valid_dataset, valid_loader, valid_path = load_dataset_loader(downstream_task, 'valid', tokenizer)
test_dataset, test_loader, test_path = load_dataset_loader(downstream_task, 'test', tokenizer)
# print(model.cuda())
finetuned_model = fine_tuning_model(model, i2w, train_loader, valid_loader, finetune_epoch)

# exp_dataset.to_csv(os.getcwd() + r'/result/'+exp_name+".csv", index=False)

INFO:absl:Using /tmp/tfhub_modules to cache modules.
2022-03-04 12:06:10.315526: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib:/usr/local/cuda/lib
2022-03-04 12:06:10.316279: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-03-04 12:06:11.209366: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensor

(Epoch 1) TRAIN LOSS:0.3491 ACC:0.87 F1:0.82 REC:0.79 PRE:0.85 LR:0.00000300


VALID LOSS:0.1921 ACC:0.93 F1:0.90 REC:0.89 PRE:0.90: 100%|█████████| 40/40 [00:03<00:00, 11.08it/s]

(Epoch 1) VALID LOSS:0.1921 ACC:0.93 F1:0.90 REC:0.89 PRE:0.90


In [8]:
exp_dataset = valid_dataset.load_dataset(valid_path).head(num_sample)
exp_dataset

,text,sentiment
0,"meski masa kampanye sudah selesai , bukan berati habis pula upaya mengerek tingkat kedipilihan elektabilitas .",1
1,tidak enak,2


In [9]:
text,label = None,None

# print(perturbation_technique)
if downstream_task == 'sentiment':
    text = 'text'
    label = 'sentiment'
    exp_dataset[['perturbed_text', 'perturbed_semantic_sim', 'pred_label', 'pred_proba', 'perturbed_label', 'perturbed_proba', 'translated_word(s)', 'running_time(s)']] = exp_dataset.apply(
        lambda row: attack(
            text_ls = row.text,
            true_label = row.sentiment,
            predictor = finetuned_model,
            tokenizer = tokenizer, 
            att_ratio = perturb_ratio,
            perturbation_technique = perturbation_technique,
            lang_codemix = perturb_lang,
            sim_predictor = use), axis=1, result_type='expand'
    )
elif downstream_task == 'emotion':
    text = 'tweet'
    label = 'label'
    exp_dataset[['perturbed_text', 'perturbed_semantic_sim', 'pred_label', 'pred_proba', 'perturbed_label', 'perturbed_proba', 'translated_word(s)', 'running_time(s)']] = exp_dataset.apply(
        lambda row: attack(
            text_ls = row.tweet,
            true_label = row.label,
            predictor = finetuned_model,
            tokenizer = tokenizer, 
            att_ratio = perturb_ratio,
            perturbation_technique = perturbation_technique,
            lang_codemix = perturb_lang,
            sim_predictor = use), axis=1, result_type='expand'
    )

before_attack = accuracy_score(exp_dataset[label], exp_dataset['pred_label'])
after_attack = accuracy_score(exp_dataset[label], exp_dataset['perturbed_label'])

exp_dataset.loc[exp_dataset.index[0], 'before_attack_acc'] = before_attack
exp_dataset.loc[exp_dataset.index[0], 'after_attack_acc'] = after_attack
exp_dataset.loc[exp_dataset.index[0], 'avg_running_time(s)'] = exp_dataset["running_time(s)"].mean()

ic| type(perturbed_text): <class 'str'>
ic| type(perturbed_semantic_sim): <class 'jaxlib.xla_extension.DeviceArray'>
ic| type(orig_label): <class 'int'>
ic| type(orig_probs): <class 'numpy.ndarray'>
ic| type(perturbed_label): <class 'int'>
ic| type(perturbed_probs): <class 'numpy.ndarray'>
ic| type(translated_words): <class 'dict'>
ic| type(running_time): <class 'float'>
ic| type(perturbed_text): <class 'str'>
ic| type(perturbed_semantic_sim): <class 'jaxlib.xla_extension.DeviceArray'>
ic| type(orig_label): <class 'int'>
ic| type(orig_probs): <class 'numpy.ndarray'>
ic| type(perturbed_label): <class 'int'>
ic| type(perturbed_probs): <class 'numpy.ndarray'>
ic| type(translated_words): <class 'dict'>
ic| type(running_time): <class 'float'>


In [15]:
main(
    model_target="IndoBERT",
    downstream_task="sentiment",
    attack_strategy="adversarial",
    finetune_epoch=1,
    num_sample=2,
    exp_name="fixx",
    perturbation_technique="codemixing",
    perturb_ratio=0.4,
    perturb_lang="en",
)

TypeError: 'module' object is not callable

In [4]:
txt = "betapa bejad kader gerindra yang anggota dprd mencabuli anak smp , rakyat harus cerdas partai mana yang harus di tengelamkan di 2019"
true_label = 2

perturbed_text, perturbed_semantic_sim, pred_label, pred_proba, perturbed_label, perturbed_proba, translated_words, running_time = attack(
    text_ls = txt,
    true_label = true_label,
    predictor = finetuned_model,
    tokenizer = tokenizer, 
    att_ratio = 0.8,
    perturbation_technique = perturbation_technique,
    lang_codemix = perturb_lang,
    sim_predictor = use
)

ic(perturbed_text)
ic(perturbed_semantic_sim)
ic(pred_label)
ic(pred_proba)
ic(perturbed_label)
ic(perturbed_proba)
ic(translated_words)
ic(running_time)

/home/m13518040/tugas-akhir-repository/attack/adv_attack.py:68: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  leave_1_probs = torch.tensor(leave_1_probs).to("cuda:1")
INFO:absl:Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker: 
INFO:absl:Unable to initialize backend 'gpu': NOT_FOUND: Could not find registered platform with name: "cuda". Available platform names are: Host Interpreter
INFO:absl:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
ic| type(perturbed_text): <class 'str'>
ic| type(perturbed_semantic_sim): <class 'jaxlib.xla_extension.DeviceArray'>
ic| type(orig_label): <class 'int'>
ic| type(orig_probs): <class 'numpy.ndarray'>
ic| type(perturbed_label): <class 'int'>
ic| type(pert

9.95

In [10]:
exp_dataset

,text,sentiment,perturbed_text,perturbed_semantic_sim,pred_label,pred_proba,perturbed_label,perturbed_proba,translated_word(s),running_time(s),before_attack_acc,after_attack_acc,avg_running_time(s)
0,"meski masa kampanye sudah selesai , bukan berati habis pula upaya mengerek tingkat kedipilihan elektabilitas .",1,meski masa campaign sudah finished bukan berati habis pula upaya mengerek level kedipilihan elektabilitas,0.8135486,1,"[0.0617, 0.5557, 0.3826]",2,"[0.0985, 0.4105, 0.491]","{'tingkat': 'level', 'kampanye': 'campaign', 'selesai': 'finished'}",4.46,1.0,0.5,4.145
1,tidak enak,2,tidak nice,0.5409587,2,"[0.0133, 0.0083, 0.9784]",2,"[0.0247, 0.0091, 0.9663]",{'enak': 'nice'},3.83,NaN,NaN,NaN


In [164]:
exp_dataset.to_csv(os.getcwd() + r'/result/'+"test"+".csv", index=False)